<h1> DRL Robot Navigation with ROS2 - Tutorial </h1>
<h4> Written by Mark Wanis & Jaran Mann, Fall 2025 </h4>

<h2> <b>Introduction</b> </h2>
<p>The goal of this project is developing a strong foundation in being able to manipulate every aspect of the Turtlebot. Once a solid foundation is developed, it aims to use machine learning to navigate the Turtlebot around obsticles towards its destination in real time.  We aim to use online simulations, via Gazebo, prior to implementing machine learning into the physical robots.  Through proper testing and optimizing, the new functions will allow for a more advanced level of artificial intelligence in the turtlebots. </p>

<h2><b>Turtlebot/ROS Basic Operations</b></h2>
<p>Always refer to this website if there are any questions regarding setups and operations: <a href="https://emanual.robotis.com/docs/en/platform/turtlebot3/quick-start/">https://emanual.robotis.com/docs/en/platform/turtlebot3/quick-start/</a></p>
<h3>Creating Workspaces</h3>
<h4>Create a new workspace:</h4>
<pre><code>mkdir -p ~/colcon_ws/src</code></pre>
<h4>Add a project to the workspace:</h4>
<pre><code>ros2 pkg create --build-type ament_python turtlelab5
cd turtlelab5/turtlelab5
wget https://raw.githubusercontent.com/ROBOTIS-GIT/turtlebot3/humble-devel/turtlebot3_example/turtlebot3_example/turtlebot3_obstacle_detection/main.py
wget https://raw.githubusercontent.com/ROBOTIS-GIT/turtlebot3/humble-devel/turtlebot3_example/turtlebot3_example/turtlebot3_obstacle_detection/turtlebot3_obstacle_detection.py</code></pre>

<h3>Connecting to/configuring Turtlebots</h3>
<p>Here are some main commands and information that will be needed.</p>
<p style="text-indent: 2em;">The address of Turtlebot 53: MAC:9D / IP:192.168.2.2 (the IP address changes)</p>

<p>To launch the Turtlebot, type the following commands in the terminal:</p>
<pre><code>ssh ubuntu@{IP_ADDRESS}
ros2 launch turtlebot3_bringup robot.launch.py
</code></pre>

<p>After launching the Turtlebot, open a new terminal and run these commands to connect with the Turtlebot:</p>
<pre><code>export ROS_DOMAIN_ID={turtlebot ID} (our turtlebot ID was 51)
export RMW_IMPLEMENTATION=rmw_fastrtps_cpp
export TURTLEBOT3_MODEL=burger
</code></pre>

<p>To build or update the workspace:</p>
<pre><code>cd to back to colcon_ws
colcon build --packages-select turtlelab5
source install/setup.bash
</code></pre>

<p>Programs to run:</p>
<pre><code>ros2 run turtlebot3_teleop teleop_keyboard
ros2 run turtlelab5 go
</code></pre>

<p>Before running any research/testing code, it is a good habit to run the teleop keyboard first.  This is to ensure that you are actually connected to the turtlebot.</p>

<h2><b>Basics of Turtlebot Publishers and Subscribers</b></h2>
<p>Here are the basics of getting comfortable with manipulating both the inputs and outputs of the robot: <a href="researchproject.py">researchproject.py</a></p>
<h3><b>Publishers:</b></h3>
<ul>
    <li>Command Velocity</li>
    <ul>
        <li>Linear Velocity</li>
        <li>Angular Velocity</li>
    </ul>
    
</ul>
<h3><b>Subscribers:</b></h3>
<ul>
    <li>Lidar</li>
    <li>Odometry</li>
    <ul>
        <li>Position </li>
    </ul>
    <li>JointState</li>
    <ul>
        <li>Velocity </li>
    </ul>
    <li>Gyroscope</li>
    <ul>
        <li>Heading</li>
    </ul>
</ul>

<h2><b>Package Installations</b></h2>
<p>Here are the packages we installed, either for convenience or because the project needed them.</p>
<h3>Optional Packages:</h3>
<ul>
  <li>
    <p><b>VS Code</b></p>
    <pre><code>sudo snap install code --classic
    </code></pre>
  </li>
  <li>
    <p><b>Jupyter Notebook</b></p>
    <pre><code>sudo apt update
pip install jupyter
    </code></pre>
  </li>
  <li>
    <p><b>GitHub Desktop</b></p>
    <pre><code>wget -qO - https://mirror.mwt.me/shiftkey-desktop/gpgkey | gpg --dearmor | sudo tee /etc/apt/keyrings/mwt-desktop.gpg > /dev/null
echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/mwt-desktop.gpg] https://mirror.mwt.me/shiftkey-desktop/deb/ any main" | sudo tee /etc/apt/sources.list.d/mwt-desktop.list
sudo apt update
sudo apt install github-desktop
    </code></pre>
  </li>
</ul>

<h3>Gazebo Dependencies:</h3>
<pre><code>sudo apt update
sudo apt install -y \
  ros-humble-rclpy \
  ros-humble-nav-msgs \
  ros-humble-geometry-msgs \
  ros-humble-sensor-msgs \
  ros-humble-gazebo-ros-pkgs \
  ros-humble-cartographer \
  ros-humble-cartographer-ros \ 
  ros-humble-turtlebot3-cartographer</pre></code>

<h3>Python Dependencies:</h3>
<pre><code>pip install --upgrade pip setuptools wheel
pip install \
  numpy \
  torch torchvision torchaudio \
  tensorboard \
  squaternion \
  stable-baselines3 \
  gym \
  matplotlib \
  pandas</pre></code>

<h2><b>Getting a Virtual Map Using Cartographer and Gazebo</b></h2>
<p>Follow these steps to set up the environment and generate a virtual map:</p>
<ol>
  <li>
    <p>(In a terminal) Open tele-op:</p>
    <pre><code>export TURTLEBOT3_MODEL=burger
ros2 run turtlebot3_teleop teleop_keyboard</code></pre>
  </li>

  <li>
    <p>(In a different terminal) Open Gazebo:</p>
    <pre><code>export TURTLEBOT3_MODEL=burger
ros2 launch turtlebot3_gazebo turtlebot3_world.launch.py</code></pre>
  </li>

  <li>
    <p>(In a different terminal) Open Cartographer:</p>
    <pre><code>export TURTLEBOT3_MODEL=burger
ros2 launch turtlebot3_cartographer cartographer.launch.py use_sim_time:=True</code></pre>
  </li>
</ol>
